In [1]:
# Importation des modules

import pandas as pd
from gensim.models import Word2Vec
import random
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import datetime

#Affichage de toutes les colonnes

pd.set_option('display.max_columns', 500)

In [2]:
w2v_model_all = Word2Vec.load('results/word2vec_all_300.model')

In [3]:
df0 = pd.read_csv("data/data_cleaned_NLP.csv", sep = ',', encoding = 'latin-1')

In [4]:
df = df0[['Date', 'Groupe', 'Orateur', 'tokenized_replique']].copy()

In [5]:
df['days'] = df.apply(lambda row : (datetime.date(int(row.Date[:4]),int(row.Date[5:7]),int(row.Date[8:])) - 
                    datetime.date(2017,6,28)).days, axis = 1)

In [6]:
def add_groupe(L, etiquette):
    return [i + etiquette for i in L]

In [ ]:
def calcule_WMD_intervalle(d1, d2):
    print('JOURS : ', d1, ' ', d2)
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Novice = df1[df1['Groupe'] == 'Novice']
    df_Exp = df1[df1['Groupe'] == 'Exp']
    
    #display(df_Novice)
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].sample(50).dropna()
        df_Exp = df1[df1['Groupe'] == 'Exp'].sample(50).dropna()
        print(df_Exp.shape, df_Novice.shape)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    

    for i in phrases_Novice:
        value = np.mean([w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp')) for j in phrases_Exp])
        print(value)
        if str(value) == 'inf':
            print(i)
        WMD.append(value)
    
    px.line([np.mean(WMD[:i]) for i in range(1, len(WMD))]).show()
    
    return np.mean(WMD)

In [ ]:
#calcule_WMD_intervalle(322, 336)

In [ ]:
#L = [calcule_WMD_intervalle(i, i + 14) for i in range(0, 1094, 14)]
#px.line(y = L, x = [i for i in range(0, 1094, 14)])

values = list((pd.read_csv('results/WMDinter_temporel.csv')['0']))
dates = list((pd.read_csv('results/WMDinter_temporel.csv')['Unnamed: 0']))

In [ ]:
#pd.DataFrame(L, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel0.csv')

In [ ]:
px.line(y = values, x = dates)

In [ ]:
date_label = [datetime.timedelta(days=i) + datetime.date(2017,6,28) for i in range(14, 1094, 14)]

In [ ]:
px.line(y = [np.mean(values[:i]) for i in range(1,len(values))], x = date_label, 
        labels = {'x' : 'Dates', 'y' : 'Moyenne cumulée des distances'})

On observe la moyenne cumulée des distances en fonction du temps. On constate une moyenne forte en début d'exercice, puis une relaxation pour atteindre une valeur limite.
On peut interpréter cela comme : 
- un début d'exercice où les novices et les expérimentés parlent différemment, la distance entre les deux langages est raltivement grande.
- une période d'apprentissage (du 20 septembre 2017 au 16 mai 2018) où les différences entre novices et expérimentés tendent à s'estomper (mais dans quel sens ? on s'attend à ce que les novices s'adaptent aux expérimentés, et pas forcement l'inverse), la distance diminue.
- une période de différenciation (du 16 mai 2018 au 12 décembre 2018), où la distance augmente légerement : comme si un des deux camps voulait se différencier de l'autre.
- une période stable (du 12 décembre 2018 à la fin de l'exercice).

## Analyse personnalisée député par député

In [ ]:
# Députés Novices parlant le plus (classement par nombre d'intervention)

dict(df0[df0['Groupe'] == 'Novice'].Orateur.value_counts())

In [ ]:
orateurs_novices = list(dict(df0[df0['Groupe'] == 'Novice'].Orateur.value_counts()).keys())

In [ ]:
def calcule_WMD_intervalle_personnalise(orateur, d1, d2):
    print(orateur)
    print('JOURS : ', d1, ' ', d2)
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Novice = df1[df1['Orateur'] == orateur]
    df_Exp = df1[df1['Groupe'] == 'Exp']
    
    #display(df_Novice)
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].dropna().sample(50)
        df_Exp = df1[df1['Groupe'] == 'Exp'].dropna().sample(50)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    
    for i in phrases_Novice:
        if len(i)<3:
            continue
        values = []
        for j in phrases_Exp:
            d = w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp'))
            if str(d) != 'inf':
                values.append(d)
        value = np.mean(values)
        #print(value)
        WMD.append(value)
    
    #px.line([np.mean(WMD[:i]) for i in range(1, len(WMD))]).show()
    print(np.mean(WMD))
    
    return np.mean(WMD)

### 1) Pietraszewski

In [ ]:
#L_pietraszewski = [calcule_WMD_intervalle_personnalise('laurent pietraszewski', i, i + 14) for i in range(0, 1094, 14)]
#pd.DataFrame(L_pietraszewski, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_pietraszewski.csv')

In [ ]:
px.line([np.mean(L_pietraszewski[:i]) for i in range(1,len(L_pietraszewski))])

### 2) Ruffin

In [ ]:
#L_ruffin = [calcule_WMD_intervalle_personnalise('francois ruffin', i, i + 14) for i in range(0, 1094, 14)]
#pd.DataFrame(L_ruffin, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_ruffin.csv')

In [ ]:
px.line([np.mean(L_ruffin[:i]) for i in range(1,len(L_ruffin))])

In [ ]:
for i in range(len(L_ruffin)):
    if str(L_ruffin[i]) == 'nan':
        L_ruffin[i] = L_ruffin[i-1]
    elif str(L_ruffin[i]) == 'inf':
        print(i, [(j, j+14) for j in range(0, 1094, 14)][i])

In [ ]:
L_ruffin

#### Autres députés

In [ ]:
orateurs_novices

In [ ]:
for orateur in orateurs_novices[2:11]:
    T = [calcule_WMD_intervalle_personnalise(orateur, i, i + 14) for i in range(0, 1094, 14)]
    pd.DataFrame(T, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_' + orateur + '.csv')


In [ ]:
for orateur in orateurs_novices[11:]:
    T = [calcule_WMD_intervalle_personnalise(orateur, i, i + 14) for i in range(0, 1094, 14)]
    pd.DataFrame(T, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_' + orateur + '.csv')

v

### En particulier pour les expérimentés

In [7]:
# Députés Novices parlant le plus (classement par nombre d'intervention)

dict(df0[df0['Groupe'] == 'Exp'].Orateur.value_counts())

{'francois de rugy': 21451,
 'richard ferrand': 16837,
 'daniele obono': 7033,
 'sebastien jumel': 6708,
 'ugo bernalicis': 6611,
 'joel giraud': 6441,
 'eric coquerel': 6264,
 'thibault bazin': 6222,
 'pierre dharreville': 6118,
 'jean luc melenchon': 5957,
 'emmanuelle menard': 5635,
 'jean paul lecoq': 5525,
 'fabien di filippo': 5485,
 'olivier veran': 5047,
 'stephane peu': 5045,
 'charles de courson': 4682,
 'veronique louwagie': 4576,
 'pierre cordier': 4545,
 'valerie rabault': 4427,
 'marc le fur': 4114,
 'caroline fiat': 4039,
 'andre chassaigne': 3917,
 'alexis corbiere': 3822,
 'jean paul dufregne': 3625,
 'patrick hetzel': 3556,
 'philippe vigier': 3508,
 'philippe gosselin': 3462,
 'gilles lurton': 3401,
 'boris vallaud': 3287,
 'raphael schellenberger': 3227,
 'francois pupponi': 3221,
 'mathilde panot': 3200,
 'fabien roussel': 3175,
 'eric woerth': 3161,
 'marie christine dalloz': 3010,
 'thierry benoit': 2937,
 'dominique potier': 2935,
 'erwan balanant': 2919,
 'chri

In [8]:
orateurs_exp = list(dict(df0[df0['Groupe'] == 'Exp'].Orateur.value_counts()).keys())

In [9]:
def calcule_WMD_intervalle_personnalise2(orateur, d1, d2):
    print(orateur)
    print('JOURS : ', d1, ' ', d2)
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Exp = df1[df1['Orateur'] == orateur]
    df_Novice = df1[df1['Groupe'] == 'Novice']
    
    #display(df_Novice)
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].dropna().sample(50)
        df_Exp = df1[df1['Groupe'] == 'Exp'].dropna().sample(50)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    
    for i in phrases_Novice:
        if len(i)<3:
            continue
        values = []
        for j in phrases_Exp:
            d = w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp'))
            if str(d) != 'inf':
                values.append(d)
        value = np.mean(values)
        #print(value)
        WMD.append(value)
    
    #px.line([np.mean(WMD[:i]) for i in range(1, len(WMD))]).show()
    print(np.mean(WMD))
    
    return np.mean(WMD)

In [10]:
for orateur in orateurs_exp:
    T = [calcule_WMD_intervalle_personnalise2(orateur, i, i + 14) for i in range(0, 1094, 14)]
    pd.DataFrame(T, [i for i in range(0, 1094, 14)]).to_csv('results/WMDinter_temporel_' + orateur + '_exp.csv')



francois de rugy
JOURS :  0   14
1.5064353675673718
francois de rugy
JOURS :  14   28
1.501538414539674
francois de rugy
JOURS :  28   42
1.5107296465627504
francois de rugy
JOURS :  42   56
1.4995175376938565
francois de rugy
JOURS :  56   70
francois de rugy
JOURS :  70   84
francois de rugy
JOURS :  84   98
1.5180284034226998
francois de rugy
JOURS :  98   112
1.4979483992067237
francois de rugy
JOURS :  112   126
1.496968482647688
francois de rugy
JOURS :  126   140
1.5043043816452468
francois de rugy
JOURS :  140   154
1.5100453892943921
francois de rugy
JOURS :  154   168
1.5101960783261368
francois de rugy
JOURS :  168   182
1.4973642406381957
francois de rugy
JOURS :  182   196
francois de rugy
JOURS :  196   210
1.4964811884518927
francois de rugy
JOURS :  210   224
1.500976655165418
francois de rugy
JOURS :  224   238
1.497665004457215
francois de rugy
JOURS :  238   252
1.4938552192893337
francois de rugy
JOURS :  252   266
1.4924733749836163
francois de rugy
JOURS :  266   

1.5021163865904699
daniele obono
JOURS :  14   28
1.5044470527028455
daniele obono
JOURS :  28   42
1.5352401610785043
daniele obono
JOURS :  42   56
1.5013175653199944
daniele obono
JOURS :  56   70
daniele obono
JOURS :  70   84
daniele obono
JOURS :  84   98
1.5056721243879228
daniele obono
JOURS :  98   112
1.5031956453000936
daniele obono
JOURS :  112   126
1.499079836150496
daniele obono
JOURS :  126   140
1.5060811214379755
daniele obono
JOURS :  140   154
1.5010330961384348
daniele obono
JOURS :  154   168
1.5032760393531748
daniele obono
JOURS :  168   182
1.5057226740846354
daniele obono
JOURS :  182   196
daniele obono
JOURS :  196   210
1.4978627615244176
daniele obono
JOURS :  210   224
1.4985654222648568
daniele obono
JOURS :  224   238
1.5010141755262023
daniele obono
JOURS :  238   252
1.4814238827905757
daniele obono
JOURS :  252   266
1.4917406488607974
daniele obono
JOURS :  266   280
1.498303209700984
daniele obono
JOURS :  280   294
1.500706653341231
daniele obono


1.4951279793170424
ugo bernalicis
JOURS :  98   112
1.493132532049191
ugo bernalicis
JOURS :  112   126
1.4986605498711816
ugo bernalicis
JOURS :  126   140
1.5040205715536872
ugo bernalicis
JOURS :  140   154
1.5137496344094
ugo bernalicis
JOURS :  154   168
1.5010375849769926
ugo bernalicis
JOURS :  168   182
1.4929165633077124
ugo bernalicis
JOURS :  182   196
ugo bernalicis
JOURS :  196   210
1.4934087167884293
ugo bernalicis
JOURS :  210   224
1.504065805468138
ugo bernalicis
JOURS :  224   238
1.4934129698036427
ugo bernalicis
JOURS :  238   252
1.4880865186222076
ugo bernalicis
JOURS :  252   266
1.4993460750269456
ugo bernalicis
JOURS :  266   280
1.4868308029823467
ugo bernalicis
JOURS :  280   294
1.5136365873804751
ugo bernalicis
JOURS :  294   308
1.5025179261236639
ugo bernalicis
JOURS :  308   322
1.4883520521233156
ugo bernalicis
JOURS :  322   336
1.4999007113433704
ugo bernalicis
JOURS :  336   350
1.4979642118066878
ugo bernalicis
JOURS :  350   364
1.4976378681154934

/Users/lilian/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/lilian/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan
ugo bernalicis
JOURS :  1092   1106
1.495950546207274
joel giraud
JOURS :  0   14
1.5020876716834646
joel giraud
JOURS :  14   28
1.5153974507837573
joel giraud
JOURS :  28   42
1.5055745494094286
joel giraud
JOURS :  42   56
1.5052266866301323
joel giraud
JOURS :  56   70
joel giraud
JOURS :  70   84
joel giraud
JOURS :  84   98
1.50206702223696
joel giraud
JOURS :  98   112
1.5067614337298594
joel giraud
JOURS :  112   126
1.5055617429659078
joel giraud
JOURS :  126   140
1.4952532109897239
joel giraud
JOURS :  140   154
1.5018918680724698
joel giraud
JOURS :  154   168
1.5138457598471202
joel giraud
JOURS :  168   182
1.4948493256782411
joel giraud
JOURS :  182   196
joel giraud
JOURS :  196   210
1.5016506483843888
joel giraud
JOURS :  210   224
1.495522357496658
joel giraud
JOURS :  224   238
1.491853758144234
joel giraud
JOURS :  238   252
1.492585245591605
joel giraud
JOURS :  252   266
1.502126298716103
joel giraud
JOURS :  266   280
1.4930366662663748
joel giraud
JOURS :  

1.506824125242685
thibault bazin
JOURS :  154   168
1.496398306052291
thibault bazin
JOURS :  168   182
1.4886574262901937
thibault bazin
JOURS :  182   196
thibault bazin
JOURS :  196   210
1.4809523238027813
thibault bazin
JOURS :  210   224
1.505964694131875
thibault bazin
JOURS :  224   238
1.4961457516423917
thibault bazin
JOURS :  238   252
1.4764340442357151
thibault bazin
JOURS :  252   266
1.4966707595121023
thibault bazin
JOURS :  266   280
1.4879439426327565
thibault bazin
JOURS :  280   294
1.5009545613217792
thibault bazin
JOURS :  294   308
1.5075559986866276
thibault bazin
JOURS :  308   322
1.502072083559708
thibault bazin
JOURS :  322   336
1.4901404069948543
thibault bazin
JOURS :  336   350
1.5057967141348008
thibault bazin
JOURS :  350   364
1.499951171101398
thibault bazin
JOURS :  364   378
1.4987460557730703
thibault bazin
JOURS :  378   392
1.5138895845229503
thibault bazin
JOURS :  392   406
1.5042980772882593
thibault bazin
JOURS :  406   420
thibault bazin
JO

KeyboardInterrupt: 

## Nombre de mots utilisés selon la période

In [ ]:
def calcule_nb_mots_periode(d1, d2):
    print('JOURS : ', d1, ' ', d2)
    df1 = df[df['days'] >= d1]
    df1 = df1[df1['days'] < d2]
    
    df_Novice = df1[df1['Orateur'] == 'Novice']
    
    
    
    
    #display(df_Novice)
    try:
        df_Novice = df1[df1['Groupe'] == 'Novice'].dropna().sample(50)
        df_Exp = df1[df1['Groupe'] == 'Exp'].dropna().sample(50)
        print(df_Exp.shape, df_Novice.shape)
    except:
        return np.nan
    
    phrases_Exp = [df_Exp.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Exp.shape[0])]
    phrases_Novice = [df_Novice.iloc[i]['tokenized_replique'].split(' ') for i in range(df_Novice.shape[0])]
    
    WMD = []
    

    for i in phrases_Novice:
        value = np.mean([w2v_model_all.wv.wmdistance(add_groupe(i, '_Novice'), 
                                                     add_groupe(j, '_Exp')) for j in phrases_Exp])
        print(value)
        if str(value) == 'inf':
            print(i)
        WMD.append(value)
    
    px.line([np.mean(WMD[:i]) for i in range(1, len(WMD))]).show()
    
    return np.mean(WMD)